In [1]:
import os
import signal
import sys
import time

import example_robot_data
import numpy as np
import pinocchio as pin

import crocoddyl
from crocoddyl.utils.quadruped import SimpleQuadrupedalGaitProblem, plotSolution
from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer


In [2]:
# Load Robot Model
# Load the GO2 robot model from example_robot_data
robot = example_robot_data.load("go2")
robot_model = robot.model
robot_data = robot_model.createData()

In [3]:
# Define Problem Parameters
TIME_STEP = 0.02
HORIZON_LENGTH = 100  # 2 seconds of simulation

# Gait Parameters for a trot
GAIT_PERIOD = 0.4  # seconds
STEPS_PER_PERIOD = int(GAIT_PERIOD / TIME_STEP)
SWING_STEPS = int(STEPS_PER_PERIOD / 2)
STANCE_STEPS = int(STEPS_PER_PERIOD / 2)

# Cost function weights, inspired by the provided snippet
# Note: reward = -cost
W_VEL = 1.0
W_ANG_VEL = 1.0
W_UPRIGHT = 0.5
W_YAW = 0.3
W_HEIGHT = 1.0
W_GAIT = 0.1
W_CTRL = 0.002 # reward_energy weight was 0.0, using a small value for regularization

# Frame IDs
foot_frame_names = ['FL_foot', 'FR_foot', 'RL_foot', 'RR_foot']
foot_frame_ids = [robot_model.getFrameId(name) for name in foot_frame_names]
base_frame_id = robot_model.getFrameId("base")
# Initial state (standing pose)
q0 = robot.q0.copy()
v0 = pin.utils.zero(robot_model.nv)
x0 = np.concatenate([q0, v0])

# Update data structure for initial state
pin.forwardKinematics(robot_model, robot_data, q0)
pin.updateFramePlacements(robot_model, robot_data)

In [4]:
viz = MeshcatVisualizer(robot)
viz.display(q0)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/


In [5]:
viz.viewer.jupyter_cell()

In [6]:
import numpy as np
import pinocchio
import crocoddyl


initial_foot_positions = {
    name: robot_data.oMf[frame_id].translation.copy()
    for name, frame_id in zip(foot_frame_names, foot_frame_ids)
}

# --- 2. Define the Optimal Control Problem ---

# Actuation model for a floating-base robot
state = crocoddyl.StateMultibody(robot_model)
actuation = crocoddyl.ActuationModelFloatingBase(state)

# Create a list of running action models
running_models = []

# Target velocity and height definitions
target_base_velocity = pin.Motion(np.array([0.3, 0.0, 0.0, 0.0, 0.0, 0.0]))
target_base_height = robot_data.oMi[base_frame_id].translation[2]

# Create a sequence of action models for the trotting gait
for t in range(HORIZON_LENGTH):
    # --- Define Costs for this time step ---
    runningCostModel = crocoddyl.CostModelSum(state, actuation.nu)

    # Cost 1: Base velocity tracking
    vel_residual = crocoddyl.ResidualModelFrameVelocity(state, base_frame_id, target_base_velocity, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED, actuation.nu)
    vel_weights = np.array([W_VEL] * 3 + [W_ANG_VEL] * 3)
    vel_cost = crocoddyl.CostModelResidual(state, crocoddyl.ActivationModelWeightedQuad(vel_weights**2), vel_residual)
    runningCostModel.addCost("base_velocity", vel_cost, 1.0)

    # Cost 2: Orientation tracking
    target_base_rotation = pin.rpy.rpyToMatrix(0, 0, 0)
    orientation_residual = crocoddyl.ResidualModelFrameRotation(state, base_frame_id, target_base_rotation, actuation.nu)
    orientation_weights = np.array([W_UPRIGHT, W_UPRIGHT, W_YAW])
    orientation_cost = crocoddyl.CostModelResidual(state, crocoddyl.ActivationModelWeightedQuad(orientation_weights**2), orientation_residual)
    runningCostModel.addCost("base_orientation", orientation_cost, 1.0)

    # Cost 3: Base height tracking
    target_base_translation = np.array([0., 0., target_base_height])
    height_residual = crocoddyl.ResidualModelFrameTranslation(state, base_frame_id, target_base_translation, actuation.nu)
    height_activation = crocoddyl.ActivationModelWeightedQuad(np.array([0., 0., W_HEIGHT])**2)
    height_cost = crocoddyl.CostModelResidual(state, height_activation, height_residual)
    runningCostModel.addCost("base_height", height_cost, 1.0)

    # Cost 4: Control regularization
    ctrl_residual = crocoddyl.ResidualModelControl(state, actuation.nu)
    ctrl_cost = crocoddyl.CostModelResidual(state, ctrl_residual)
    runningCostModel.addCost("control_reg", ctrl_cost, W_CTRL)
    
    # --- Define Contacts and Swing Foot Costs for the Gait ---
    contacts = crocoddyl.ContactModelMultiple(state, actuation.nu)
    time_in_cycle = t % STEPS_PER_PERIOD
    is_first_pair_swinging = time_in_cycle < SWING_STEPS
    
    swing_foot_names, stance_foot_names = [], []
    swing_foot_ids, stance_foot_ids = [], []

    if is_first_pair_swinging:
        swing_foot_ids.extend([foot_frame_ids[0], foot_frame_ids[3]])
        swing_foot_names.extend([foot_frame_names[0], foot_frame_names[3]])
        stance_foot_ids.extend([foot_frame_ids[1], foot_frame_ids[2]])
        stance_foot_names.extend([foot_frame_names[1], foot_frame_names[2]])
    else:
        swing_foot_ids.extend([foot_frame_ids[1], foot_frame_ids[2]])
        swing_foot_names.extend([foot_frame_names[1], foot_frame_names[2]])
        stance_foot_ids.extend([foot_frame_ids[0], foot_frame_ids[3]])
        stance_foot_names.extend([foot_frame_names[0], foot_frame_names[3]])
        
    # Add 3D contact models for all stance feet
    for i, foot_id in enumerate(stance_foot_ids):
        # --- FIX: Add the missing pinocchio.ReferenceFrame.LOCAL argument ---
        contact_model = crocoddyl.ContactModel3D(state, foot_id, np.array([0., 0., 0.]), pinocchio.ReferenceFrame.LOCAL, actuation.nu)
        contacts.addContact(stance_foot_names[i] + "_contact", contact_model)

    # Cost 5: Swing foot trajectory
    if is_first_pair_swinging:
        swing_phase = float(time_in_cycle) / SWING_STEPS
    else:
        swing_phase = float(time_in_cycle - SWING_STEPS) / SWING_STEPS
    
    step_height = 0.05
    target_foot_z_offset = step_height * np.sin(swing_phase * np.pi)

    for i, foot_id in enumerate(swing_foot_ids):
        foot_name = swing_foot_names[i]
        initial_pos = initial_foot_positions[foot_name]
        target_foot_pos = initial_pos.copy()
        target_foot_pos[2] += target_foot_z_offset

        foot_track_residual = crocoddyl.ResidualModelFrameTranslation(state, foot_id, target_foot_pos, actuation.nu)
        foot_track_cost = crocoddyl.CostModelResidual(state, foot_track_residual)
        runningCostModel.addCost(foot_name + "_swing", foot_track_cost, W_GAIT)

    # --- Create the action model for this time step ---
    dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(state, actuation, contacts, runningCostModel)
    model = crocoddyl.IntegratedActionModelEuler(dmodel, TIME_STEP)
    running_models.append(model)

# --- 3. Define the Terminal Model ---
terminalCostModel = crocoddyl.CostModelSum(state, actuation.nu)

# Terminal costs for final state (robot is stationary)
terminal_vel_residual = crocoddyl.ResidualModelFrameVelocity(state, base_frame_id, pin.Motion.Zero(), pin.ReferenceFrame.LOCAL_WORLD_ALIGNED, actuation.nu)
terminal_vel_cost = crocoddyl.CostModelResidual(state, terminal_vel_residual)
terminalCostModel.addCost("terminal_velocity", terminal_vel_cost, 10 * W_VEL) 

# Regularize terminal state towards the initial state x0
state_residual = crocoddyl.ResidualModelState(state, x0, actuation.nu)
state_cost = crocoddyl.CostModelResidual(state, state_residual)
terminalCostModel.addCost("terminal_state", state_cost, 1.0)

# Terminal contact on all feet
terminal_contacts = crocoddyl.ContactModelMultiple(state, actuation.nu)
for i, foot_id in enumerate(foot_frame_ids):
    # --- FIX: Add the missing pinocchio.ReferenceFrame.LOCAL argument here as well ---
    contact_model = crocoddyl.ContactModel3D(state, foot_id, np.array([0, 0, 0]), pinocchio.ReferenceFrame.LOCAL, actuation.nu)
    terminal_contacts.addContact(foot_frame_names[i] + "_contact", contact_model)

terminal_dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(state, actuation, terminal_contacts, terminalCostModel)
terminal_model = crocoddyl.IntegratedActionModelEuler(terminal_dmodel, 0.0)


In [7]:
# 4. Create the shooting problem and DDP solver
problem = crocoddyl.ShootingProblem(x0, running_models, terminal_model)
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

# 5. Solve the OCP
print("--- Solving Go2 Walking OCP ---")
# Use warm-start trajectories of the initial state and zero controls
xs_init = [x0] * (HORIZON_LENGTH + 1)
us_init = ddp.problem.quasiStatic(xs_init[:-1]) # Gravity compensation controls

is_feasible = ddp.solve(xs_init, us_init, maxiter=100)


--- Solving Go2 Walking OCP ---
iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  3.109e+01  0.000e+00  5.149e+00  1.030e+01  1.000e-09  1.000e-09  0.2500  8.585e-01  0.000e+00  0.000e+00  2.253e+00 -3.033e+01  0.000e+00  0.000e+00
   1  2.402e+01  0.000e+00  3.093e+01  6.185e+01  1.000e-09  1.000e-09  0.2500  0.000e+00  0.000e+00  0.000e+00  1.353e+01  7.065e+00  0.000e+00  0.000e+00
   2  1.857e+01  0.000e+00  2.386e+01  4.772e+01  1.000e-09  1.000e-09  0.1250  0.000e+00  0.000e+00  0.000e+00  5.592e+00  5.456e+00  0.000e+00  0.000e+00
   3  1.434e+01  0.000e+00  1.840e+01  3.681e+01  1.000e-09  1.000e-09  0.1250  0.000e+00  0.000e+00  0.000e+00  4.313e+00  4.226e+00  0.000e+00  0.000e+00
   4  1.108e+01  0.000e+00  1.418e+01  2.835e+01  1.000e-09  1.000e-09  0.1250  0.000e+00  0.000e+00  0.000e+00  3.323e+00  3.264e+00  0.000e+00  0.000e+00
   5  8.559e+00  0.000e+00  1.091e+

In [8]:
# 4. Create the shooting problem and FDDP solver
# SolverFDDP is generally more robust for contact-rich problems
problem = crocoddyl.ShootingProblem(x0, running_models, terminal_model)
fddp = crocoddyl.SolverFDDP(problem)
fddp.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

# 5. Solve the OCP
print("--- Solving Go2 Walking OCP ---")
# Use warm-start trajectories of the initial state and zero controls
xs_init = [x0] * (HORIZON_LENGTH + 1)
us_init = fddp.problem.quasiStatic(xs_init[:-1]) # Gravity compensation controls

is_feasible = fddp.solve(xs_init, us_init, maxiter=100)

--- Solving Go2 Walking OCP ---
iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  5.109e-01  0.000e+00  5.836e-01  1.352e+00  1.000e-09  1.000e-09  0.2500  8.585e-01  0.000e+00  0.000e+00  2.760e-01  2.440e-01  0.000e+00  0.000e+00
   1  3.772e-01  0.000e+00  3.488e-01  7.689e-01  1.000e-09  1.000e-09  0.2500  6.439e-01  0.000e+00  0.000e+00  1.667e-01  1.337e-01  0.000e+00  0.000e+00
   2  3.096e-01  0.000e+00  2.139e-01  4.790e-01  1.000e-09  1.000e-09  0.2500  4.829e-01  0.000e+00  0.000e+00  1.017e-01  6.760e-02  0.000e+00  0.000e+00
   3  2.771e-01  0.000e+00  1.458e-01  3.209e-01  1.000e-09  1.000e-09  0.1250  3.622e-01  0.000e+00  0.000e+00  3.687e-02  3.253e-02  0.000e+00  0.000e+00
   4  2.524e-01  0.000e+00  1.140e-01  2.499e-01  1.000e-09  1.000e-09  0.1250  3.169e-01  0.000e+00  0.000e+00  2.866e-02  2.466e-02  0.000e+00  0.000e+00
   5  2.346e-01  0.000e+00  8.969e-

In [9]:
if not is_feasible:
    print("\n--- OCP Failed to Converge ---")

else:
    print("\n--- OCP Solved Successfully ---")

print(f"Final cost: {fddp.cost}")

# You can now access the solution
xs = fddp.xs  # State trajectory
us = fddp.us  # Control trajectory

print(f"Solved states shape: {np.array(xs).shape}")
print(f"Solved controls shape: {np.array(us).shape}")

# For visualization, you would typically use a library like MeshCat
# For example:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer()
viz.loadViewerModel()
viz.display(xs[0][:robot_model.nq])



--- OCP Failed to Converge ---
Final cost: 0.16522186417440043
Solved states shape: (101, 37)
Solved controls shape: (100, 12)
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [12]:
import time
for x in xs:
    viz.display(x[:robot_model.nq])
    time.sleep(TIME_STEP)

In [11]:
viz.viewer.jupyter_cell()

In [12]:
len(robot_model.frames.tolist())

for elem in robot_model.frames:
    print(elem.name)

foot_frame_ids

universe
root_joint
base
FL_hip_joint
FL_hip
FL_thigh_joint
FL_thigh
FL_calf_joint
FL_calf
FL_foot_joint
FL_foot
FL_calf_rotor_joint
FL_calf_rotor
FL_thigh_rotor_joint
FL_thigh_rotor
FL_hip_rotor_joint
FL_hip_rotor
FR_hip_joint
FR_hip
FR_thigh_joint
FR_thigh
FR_calf_joint
FR_calf
FR_foot_joint
FR_foot
FR_calf_rotor_joint
FR_calf_rotor
FR_thigh_rotor_joint
FR_thigh_rotor
FR_hip_rotor_joint
FR_hip_rotor
RL_hip_joint
RL_hip
RL_thigh_joint
RL_thigh
RL_calf_joint
RL_calf
RL_foot_joint
RL_foot
RL_calf_rotor_joint
RL_calf_rotor
RL_thigh_rotor_joint
RL_thigh_rotor
RL_hip_rotor_joint
RL_hip_rotor
RR_hip_joint
RR_hip
RR_thigh_joint
RR_thigh
RR_calf_joint
RR_calf
RR_foot_joint
RR_foot
RR_calf_rotor_joint
RR_calf_rotor
RR_thigh_rotor_joint
RR_thigh_rotor
RR_hip_rotor_joint
RR_hip_rotor
imu_joint
imu
radar_joint
radar


[10, 24, 38, 52]